# Exercise 3 | Market Risk

Actuarial Science Undergratuate Program | Facultad de Negocios | Universidad La Salle México | Financial Risks Class | José Enrique Pérez Salvador

Based on the context of Exercise 2, you want to estimate the annual volatility of JPMORGAN CHASE & CO.'s stock listed on the NYSE (ticker JPM). You will consider the price series from August 16, 2017 to September 3rd, 2024, and the calendar of business days in Mexico.
Let $ S_t $, $ t=0,1,...,n $ the JPM stock price, let $ X_1,...,X_n $ its daily logreturns series computed as $ X_t = \ln⁡(S_t/S_{t-1})$, $ t=1,...,n $. Our goal is to model $ X_t $.


## Import

In [ ]:
proc sql;
%if %sysfunc(exist(WORK.'transform'n)) %then %do;
    drop table WORK.'transform'n;
%end;
%if %sysfunc(exist(WORK.'transform'n,VIEW)) %then %do;
    drop view WORK.'transform'n;
%end;
quit;

FILENAME REFFILE DISK '';


PROC IMPORT DATAFILE=REFFILE
	DBMS=XLSX
	OUT=WORK.'transform'n;
	GETNAMES=YES;
RUN;

PROC CONTENTS DATA=WORK.'transform'n; RUN;


 ## Exploration

In [ ]:
ods graphics / imagemap=on;

proc sort data=WORK.TRANSFORM(keep=date RetJPM) out=Work.preProcessedData;
	by date;
run;

proc timeseries data=Work.preProcessedData seasonality=7 plots=(series 
		histogram corr);
	id date interval=Day;
	var  / accumulate=none transform=none;
run;

proc delete data=Work.preProcessedData;
run;

## Model

In [ ]:
ods output FitSummary=FIT_SUMMARY ParameterEstimates = PARAMETER_ESTIMATES ;
proc autoreg data=(keep=date RetJPM) maxit=200 optimizer=nlp;
	"ARCH(p=1)": model RetJPM=/ garch=(q=) noint;
	output out=results_arch1 cev=v;
	"ARCH(p=2)": model RetJPM=/ garch=(q=) noint;
	output out=results_arch2 cev=v;
	"GARCH(p=1,q=1)": model RetJPM=/ garch=(q=,p=) noint;
	output out=results_garch_1_1 cev=v;
	"GARCH(p=1,q=2)": model RetJPM=/ garch=(q=,p=) noint;
	output out=results_garch_1_2 cev=v;
	"GARCH(p=2,q=1)": model RetJPM=/ garch=(q=,p=) noint;
	output out=results_garch_2_1 cev=v;
	"GARCH(p=2,q=2)": model RetJPM=/ garch=(q=,p=) noint;
	output out=results_garch_2_2 cev=v;
run;

## Selection the best model based on the AICC

In [ ]:
proc sql;
	select model, label2, nvalue2
	from FIT_SUMMARY
	where label2 = "AICC"
	having min(nvalue2)=nvalue2
	;
quit;

Get the estimated volatility with the champion model

In [ ]:
proc sql;
	create table work.var_cond as
	select a.date
	, a.v as variance label="Conditional variance with GARCH(1,1)"
	, (252*a.v)**0.5 as vol label="Annual volatility"
	from work.results_garch_1_1 a 
	;
quit;

Plot the estimated volatilities

In [ ]:
ods graphics / reset width=6.4in height=4.8in imagemap noborder;

proc sort data=work.var_cond out=_SeriesPlotTaskData;
	by date;
run;

proc sgplot data=_SeriesPlotTaskData;
	title height=14pt "Volatilidad";
	series x=date y=vol /;
	xaxis grid;
	yaxis grid;
run;

ods graphics / reset;
title;

proc datasets library=WORK noprint;
	delete _SeriesPlotTaskData;
run;